In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 133 kB 4.4 MB/s 


In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, LSTM, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras_tuner.tuners import BayesianOptimization
from keras.optimizer_v2 import adam as adam_v2
import os
import math
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df_org = pd.read_csv("/content/drive/MyDrive/DC2/all_variables.csv")
df_org

Mounted at /content/drive


,Year,Month,LSOA_code,LSOA_name,Type_I_crime_amount,Total_month,Type_II_crime_amount
0,2011,1,E01000001,City of London 001A,2,1,8
1,2011,2,E01000001,City of London 001A,1,2,8
2,2011,3,E01000001,City of London 001A,6,3,4
3,2011,4,E01000001,City of London 001A,1,4,4
4,2011,6,E01000001,City of London 001A,1,6,15
...,...,...,...,...,...,...,...
3286689,2021,6,W01001958,Swansea 025H,6,126,16
3286690,2021,7,W01001958,Swansea 025H,4,127,11
3286691,2021,8,W01001958,Swansea 025H,9,128,10
3286692,2021,9,W01001958,Swansea 025H,4,129,16


In [ ]:
# E01000005 for Urban
# E01030759 for Town and Fringe
# E01020795 for Village Hamlet & Isolated Dwellings

In [ ]:
# Extract the data from a given LSOA code
df_need = df_org[df_org['LSOA_code'] == 'E01030759'].reset_index(drop = True)
df_need = df_need[['Total_month', 'Type_I_crime_amount', 'Type_II_crime_amount']]
df_need = df_need[df_need['Total_month'] <= 108]
df_need.drop(columns = 'Total_month', inplace = True)
df_need

,Type_I_crime_amount,Type_II_crime_amount
0,5,20
1,6,22
2,10,19
3,5,20
4,6,28
...,...,...
103,6,8
104,8,9
105,9,12
106,11,18


In [ ]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences) - 1:
          break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i: end_ix], sequences[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
# Hyperparameter searching if necessary

def build_model(hp):
    # Build the LSTM model with 2 LSTM layers and 1 Dense layer
    input = Input(shape = (12, 2))
    l1 = LSTM(units = hp.Int('units_1', min_value = 50, max_value = 300, step = 50), return_sequences = True, dropout = 0.2)(input)
    b1 = BatchNormalization()(l1)
    l2 = LSTM(units = hp.Int('units_2', min_value = 50, max_value = 300, step = 50), return_sequences = False, dropout = 0.2)(b1)
    b2 = BatchNormalization()(l2)
    output = Dense(2)(b2)
    model = Model(input, output)

    # The optimizer is Adam
    opt = adam_v2.Adam(learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4, 2e-2, 2e-3, 2e-4]))
    model.compile(loss = 'mse', optimizer = opt, metrics = ['mse'])

    return model

def find_best_model():
    # Example of splitting the data when time step is 12 months: 
    # Training data has data from 1st month to 24th month
    # Val data has data from 13th month to 36th month
    # Test data predicts new data using data from 25th month to 36th month
    train_data = df_need.to_numpy()[: 96]
    val_data = df_need.to_numpy()[84: 108]
    # Split x and y arrays using sliding window approach
    x_train, y_train = split_sequences(train_data, 12)
    x_val, y_val = split_sequences(val_data, 12)
    bayesian_opt_tuner = BayesianOptimization(build_model, objective = 'mse',
        max_trials = 216,
        executions_per_trial = 1,
        directory = '/content/drive/MyDrive/DC2/tune/Suburban/',
        project_name = 'kerastuner_bayesian_poc',
        overwrite = True)

    bayesian_opt_tuner.search(x_train, y_train, epochs = 500, validation_data = (x_val, y_val))

    bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models = 1)
    model = bayes_opt_model_best_model[0]
    return model

#model = find_best_model()
#model.summary()

In [ ]:
def train(df, train_nr, prediction_nr, comparison, model_exist, n_steps=12):
    with tf.device(device_name):
        # Example of splitting the data when time step is 12 months: 
        # Training data has data from 1st month to 24th month
        # Val data has data from 13th month to 36th month
        # Test data predicts new data using data from 25th month to 36th month
        train_data = df.to_numpy()[: train_nr - n_steps]
        val_data = df.to_numpy()[train_nr - n_steps * 2: train_nr]
        # Split x and y arrays using sliding window approach
        x_train, y_train = split_sequences(train_data, n_steps)
        x_val, y_val = split_sequences(val_data, n_steps)

        if model_exist == "Nil":
            # Build the LSTM model with 2 LSTM layers and 1 Dense layer
            input = Input(shape = (n_steps, 2))
            l1 = LSTM(50, return_sequences = True, dropout = 0.2)(input)
            b1 = BatchNormalization()(l1)
            l2 = LSTM(50, return_sequences = False, dropout = 0.2)(b1)
            b2 = BatchNormalization()(l2)
            output = Dense(2)(b2)
            model = Model(input, output)

            # The optimizer is Nadam
            opt = adam_v2.Adam(learning_rate = 0.005, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08)

            # The loss used in model training is mean_squared_error because it is crime number prediction
            model.compile(loss = 'mse', optimizer = opt)

        else:
            model = model_exist
        
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 200)
        checkpoint_filepath = '/content/drive/MyDrive/DC2/model/Suburban/' + str(train_nr) + '/weights.{epoch:02d}.h5'
        model_checkpoint_callback = ModelCheckpoint(filepath = checkpoint_filepath, monitor = 'val_loss', mode = 'min', save_best_only = True)
        lr_reducer = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 50, verbose = 0, mode = 'auto', min_delta = 0.0001, cooldown = 0, min_lr = 0)

        # Fit the model with 1000 epoches and batch size 64
        # Validation data is used here for evaluation during the training process
        model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 1000, batch_size = 32, callbacks = [model_checkpoint_callback, lr_reducer, early_stopping])

        # Look for all file directories in the folder
        directory = "/content/drive/MyDrive/DC2/model/Urban/" + str(train_nr) + '/'
        all_file_lst = []
        for root, subdirectories, files in os.walk(directory):
            for file in files:
                all_file_lst.append(os.path.join(root, file))
        # Load the best model trained
        model = load_model(all_file_lst[-1])

        # Make predictions from testing data
        # Pred x is a list of predicted data used
        # Pred y is a list of predicted result
        # Get the last 12 results from overall training data for prediction data in pred x
        pred_x = np.array(list(val_data[-(n_steps): ]))
        pred_x = pred_x.reshape((1, pred_x.shape[0], pred_x.shape[1]))
        # Initialize pred y as empty array
        pred_y = np.empty((1, 1, 2), int)
        for i in range(prediction_nr):
            # Always round off the latest prediction to integer because number of crime can only be integer
            prediction = np.round(model.predict(pred_x))
            # Update pred y 
            pred_y = np.append(pred_y, [prediction], axis = 1)
            # Update pred x by feeding the latest prediction result as latest feature
            pred_x = np.append(pred_x, [prediction], axis = 1)
            # Update pred x by removing the 1st row because of the timestep size
            pred_x = pred_x[: , 1: , : ]
        # Remove the 1st row of pred y because when initializing array, random numbers are assigned which are not actual results
        pred_y = pred_y[: , 1: , : ]
        
        df_predict = pd.DataFrame(columns = ['Type_I_crime_amount', 'Type_II_crime_amount'])
        df_predict['Type_I_crime_amount'] = np.round(pred_y[: , : , 0])[0]
        df_predict['Type_II_crime_amount'] = np.round(pred_y[: , : , 1])[0]

        if comparison == False:
            return df_predict
        else:
            # Compute RMSE, MAE and R2 by comparing predicted result from actual crime number
            y_true = df[train_nr: (train_nr + prediction_nr)]
            rmse_1 = math.sqrt(mean_squared_error(y_true['Type_I_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist()))
            rmse_2 = math.sqrt(mean_squared_error(y_true['Type_II_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist()))
            mae_1 = mean_absolute_error(y_true['Type_I_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            mae_2 = mean_absolute_error(y_true['Type_II_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            medae_1 = median_absolute_error(y_true['Type_I_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            medae_2 = median_absolute_error(y_true['Type_II_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            r2_1 = r2_score(y_true['Type_I_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            r2_2 = r2_score(y_true['Type_II_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            return df_predict, rmse_1, rmse_2, mae_1, mae_2, medae_1, medae_2, r2_1, r2_2

In [ ]:
%%time
df_compare_predict = pd.DataFrame(columns = ['Type_I_crime_amount', 'Type_II_crime_amount'])
rmse_lst_1 = []
rmse_lst_2 = []
mae_lst_1 = []
mae_lst_2 = []
medae_lst_1 = []
medae_lst_2 = []
r2_lst_1 = []
r2_lst_2 = []

# Month Forward-Chaining Cross Validation Approach is used below
for i in tqdm(range(6)):
    df_small, rmse_1_small, rmse_2_small, mae_1_small, mae_2_small, medae_1_small, medae_2_small, r2_1_small, r2_2_small = train(df_need, 36 + 12 * i, 12, True, "Nil")
    df_compare_predict = pd.concat([df_compare_predict, df_small])
    rmse_lst_1.append(rmse_1_small)
    rmse_lst_2.append(rmse_2_small)
    mae_lst_1.append(mae_1_small)
    mae_lst_2.append(mae_2_small)
    medae_lst_1.append(medae_1_small)
    medae_lst_2.append(medae_2_small)    
    r2_lst_1.append(r2_1_small)
    r2_lst_2.append(r2_2_small)

df_compare_predict

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/1000
1/1 [==============================] - 10s 10s/step - loss: 179.3400 - val_loss: 199.6855 - lr: 0.0050
Epoch 2/1000
1/1 [==============================] - 0s 53ms/step - loss: 175.0533 - val_loss: 202.1432 - lr: 0.0050
Epoch 3/1000
1/1 [==============================] - 0s 41ms/step - loss: 178.2050 - val_loss: 200.4415 - lr: 0.0050
Epoch 4/1000
1/1 [==============================] - 0s 119ms/step - loss: 170.3201 - val_loss: 199.4628 - lr: 0.0050
Epoch 5/1000
1/1 [==============================] - 0s 119ms/step - loss: 168.4041 - val_loss: 195.3353 - lr: 0.0050
Epoch 6/1000
1/1 [==============================] - 0s 118ms/step - loss: 172.9870 - val_loss: 190.6777 - lr: 0.0050
Epoch 7/1000
1/1 [==============================] - 0s 117ms/step - loss: 170.0362 - val_loss: 187.3834 - lr: 0.0050
Epoch 8/1000
1/1 [==============================] - 0s 117ms/step - loss: 163.0543 - val_loss: 185.3425 - lr: 0.0050
Epoch 9/1000
1/1 [==============================] - 0s 43ms/step - 

 17%|█▋        | 1/6 [00:37<03:06, 37.37s/it]

Epoch 1/1000
1/1 [==============================] - 5s 5s/step - loss: 184.4612 - val_loss: 178.5139 - lr: 0.0050
Epoch 2/1000
1/1 [==============================] - 0s 120ms/step - loss: 179.6202 - val_loss: 177.1299 - lr: 0.0050
Epoch 3/1000
1/1 [==============================] - 0s 44ms/step - loss: 180.0530 - val_loss: 177.5095 - lr: 0.0050
Epoch 4/1000
1/1 [==============================] - 0s 49ms/step - loss: 179.9483 - val_loss: 178.1415 - lr: 0.0050
Epoch 5/1000
1/1 [==============================] - 0s 52ms/step - loss: 175.4760 - val_loss: 179.1038 - lr: 0.0050
Epoch 6/1000
1/1 [==============================] - 0s 49ms/step - loss: 178.4651 - val_loss: 180.2872 - lr: 0.0050
Epoch 7/1000
1/1 [==============================] - 0s 47ms/step - loss: 171.9103 - val_loss: 182.0932 - lr: 0.0050
Epoch 8/1000
1/1 [==============================] - 0s 47ms/step - loss: 172.3134 - val_loss: 183.6644 - lr: 0.0050
Epoch 9/1000
1/1 [==============================] - 0s 44ms/step - loss: 

 33%|███▎      | 2/6 [01:07<02:11, 32.86s/it]

...

Epoch 1/1000
2/2 [==============================] - 5s 1s/step - loss: 132.1663 - val_loss: 77.6827 - lr: 0.0050
Epoch 2/1000
2/2 [==============================] - 0s 113ms/step - loss: 124.0722 - val_loss: 76.0201 - lr: 0.0050
Epoch 3/1000
2/2 [==============================] - 0s 116ms/step - loss: 119.1114 - val_loss: 74.3050 - lr: 0.0050
Epoch 4/1000
2/2 [==============================] - 0s 114ms/step - loss: 119.5992 - val_loss: 73.5005 - lr: 0.0050
Epoch 5/1000
2/2 [==============================] - 0s 112ms/step - loss: 117.3336 - val_loss: 72.4927 - lr: 0.0050
Epoch 6/1000
2/2 [==============================] - 0s 121ms/step - loss: 114.5636 - val_loss: 72.0583 - lr: 0.0050
Epoch 7/1000
2/2 [==============================] - 0s 119ms/step - loss: 114.7277 - val_loss: 71.8170 - lr: 0.0050
Epoch 8/1000
2/2 [==============================] - 0s 110ms/step - loss: 110.1102 - val_loss: 71.2768 - lr: 0.0050
Epoch 9/1000
2/2 [==============================] - 0s 113ms/step - loss: 1

 83%|████████▎ | 5/6 [03:35<00:46, 46.02s/it]

Epoch 1/1000
3/3 [==============================] - 5s 621ms/step - loss: 120.4660 - val_loss: 89.0975 - lr: 0.0050
Epoch 2/1000
3/3 [==============================] - 0s 60ms/step - loss: 117.4353 - val_loss: 86.4190 - lr: 0.0050
Epoch 3/1000
3/3 [==============================] - 0s 27ms/step - loss: 112.7511 - val_loss: 88.3311 - lr: 0.0050
Epoch 4/1000
3/3 [==============================] - 0s 32ms/step - loss: 110.7059 - val_loss: 88.8536 - lr: 0.0050
Epoch 5/1000
3/3 [==============================] - 0s 27ms/step - loss: 111.0570 - val_loss: 89.1355 - lr: 0.0050
Epoch 6/1000
3/3 [==============================] - 0s 29ms/step - loss: 108.2386 - val_loss: 88.1307 - lr: 0.0050
Epoch 7/1000
3/3 [==============================] - 0s 65ms/step - loss: 104.2979 - val_loss: 86.2078 - lr: 0.0050
Epoch 8/1000
3/3 [==============================] - 0s 63ms/step - loss: 101.2946 - val_loss: 85.3316 - lr: 0.0050
Epoch 9/1000
3/3 [==============================] - 0s 69ms/step - loss: 96.417

100%|██████████| 6/6 [04:25<00:00, 44.25s/it]

CPU times: user 2min 40s, sys: 11.9 s, total: 2min 52s
Wall time: 4min 25s


In [ ]:
rmse_lst_1, rmse_lst_2

([2.3979157616563596,
  3.1754264805429417,
  4.133198922545748,
  3.6742346141747673,
  4.041451884327381,
  3.730504880933232],
 [12.829003598617211,
  4.406434688800762,
  4.092676385936225,
  3.605551275463989,
  4.222953153106642,
  5.131601439446884])

In [ ]:
mae_lst_1, mae_lst_2

([1.9166666666666667,
  2.4166666666666665,
  3.5833333333333335,
  3.1666666666666665,
  3.6666666666666665,
  3.0833333333333335],
 [9.583333333333334,
  3.0833333333333335,
  2.9166666666666665,
  3.1666666666666665,
  3.6666666666666665,
  4.166666666666667])

In [ ]:
medae_lst_1, medae_lst_2

([1.5, 2.5, 3.0, 3.0, 4.0, 2.5], [5.0, 2.0, 2.0, 3.0, 3.0, 3.0])

In [ ]:
r2_lst_1, r2_lst_2

([-1.1789473684210527,
  -0.9005235602094241,
  -2.1538461538461537,
  -1.495507060333761,
  -1.308145240431796,
  -0.8368469294225482],
 [-1.0494638533379455,
  -0.15346534653465338,
  -0.125,
  -0.4322876817138488,
  0.013446023818670882,
  -0.4720496894409938])

In [ ]:
rmse_1 = sum(rmse_lst_1) / len(rmse_lst_1)
rmse_2 = sum(rmse_lst_2) / len(rmse_lst_2)
mae_1 = sum(mae_lst_1) / len(mae_lst_1)
mae_2 = sum(mae_lst_2) / len(mae_lst_2)
medae_1 = sum(medae_lst_1) / len(medae_lst_1)
medae_2 = sum(medae_lst_2) / len(medae_lst_2)
r2_1 = sum(r2_lst_1) / len(r2_lst_1)
r2_2 = sum(r2_lst_2) / len(r2_lst_2)

print("Type I crime amount: \n RMSE: " + str(rmse_1) + "\n MAE: " + str(mae_1) + "\n MEDAE: " + str(medae_1) + "\n R2: " + str(r2_1) + 
      "\n \n Type II crime amount: \n RMSE: " + str(rmse_2) + "\n MAE: " + str(mae_2) + "\n MEDAE: " + str(medae_1) + "\n R2: " + str(r2_2))

Type I crime amount: 
 RMSE: 3.525455424030071
 MAE: 2.972222222222222
 MEDAE: 2.75
 R2: -1.312302718777456
 
 Type II crime amount: 
 RMSE: 5.714703423561953
 MAE: 4.430555555555556
 MEDAE: 2.75
 R2: -0.36980342453479514


In [ ]:
df_real_predict = train(df_need, 108, 24, False, "Nil")
df_real_predict

Epoch 1/1000
3/3 [==============================] - 5s 609ms/step - loss: 119.4707 - val_loss: 120.5092 - lr: 0.0050
Epoch 2/1000
3/3 [==============================] - 0s 26ms/step - loss: 115.6316 - val_loss: 121.4058 - lr: 0.0050
Epoch 3/1000
3/3 [==============================] - 0s 25ms/step - loss: 111.1816 - val_loss: 121.1590 - lr: 0.0050
Epoch 4/1000
3/3 [==============================] - 0s 67ms/step - loss: 107.8032 - val_loss: 120.2556 - lr: 0.0050
Epoch 5/1000
3/3 [==============================] - 0s 59ms/step - loss: 106.5468 - val_loss: 119.7677 - lr: 0.0050
Epoch 6/1000
3/3 [==============================] - 0s 62ms/step - loss: 104.7296 - val_loss: 118.4224 - lr: 0.0050
Epoch 7/1000
3/3 [==============================] - 0s 67ms/step - loss: 102.3128 - val_loss: 115.2947 - lr: 0.0050
Epoch 8/1000
3/3 [==============================] - 0s 61ms/step - loss: 99.6207 - val_loss: 110.6051 - lr: 0.0050
Epoch 9/1000
3/3 [==============================] - 0s 64ms/step - loss:

,Type_I_crime_amount,Type_II_crime_amount
0,12.0,32.0
1,8.0,34.0
2,6.0,38.0
3,7.0,43.0
4,6.0,45.0
5,5.0,44.0
6,6.0,40.0
7,7.0,40.0
8,8.0,42.0
9,7.0,42.0


In [ ]:
df_compare_predict.to_csv("/content/drive/MyDrive/DC2/suburban_old_prediction.csv", index = False)
df_real_predict.to_csv("/content/drive/MyDrive/DC2/suburban_new_prediction.csv", index = False)